# Copy Congress Strategy - Backtest Analysis

This notebook runs comprehensive backtests of the Copy Congress trading strategy.

## Objectives
1. Execute full backtest simulation
2. Calculate performance metrics
3. Compare to benchmark (SPY)
4. Analyze risk-adjusted returns
5. Visualize performance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import warnings
warnings.filterwarnings('ignore')

from data_acquisition import CongressionalDataAcquisition
from signal_generator import SignalGenerator
from portfolio_constructor import PortfolioConstructor
from backtester import CongressBacktester

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)

print('Imports complete')

## 1. Load Configuration and Data

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print('Configuration loaded')
print(f"Initial capital: ${config['backtest']['initial_capital']:,.0f}")
print(f"Benchmark: {config['backtest']['benchmark']}")

In [ ]:
# Load data
print('Loading data...')
data_acq = CongressionalDataAcquisition(config)
congressional_trades, prices, volumes, market_caps, volatility = data_acq.get_full_dataset()
print('Data loaded successfully')

## 2. Generate Signals and Construct Portfolios

In [ ]:
# Generate signals
print('Generating signals...')
signal_gen = SignalGenerator(config)

rebalance_dates = pd.date_range(
    start=prices.index[0],
    end=prices.index[-1],
    freq=config['portfolio']['rebalance_frequency']
)

signals_history = signal_gen.generate_signals_timeseries(
    congressional_trades,
    prices,
    rebalance_dates.tolist()
)

print('Signals generated')

In [ ]:
# Construct portfolios
print('Constructing portfolios...')
portfolio_constructor = PortfolioConstructor(config)

portfolio_history = portfolio_constructor.generate_portfolio_timeseries(
    signals_history,
    volatility,
    rebalance_dates.tolist()
)

print('Portfolios constructed')

## 3. Run Backtest

In [ ]:
# Run backtest
print('Running backtest...')
backtester = CongressBacktester(config)

results = backtester.simulate_portfolio(portfolio_history, prices)

print('Backtest complete')

In [ ]:
# Calculate performance metrics
metrics = backtester.calculate_performance_metrics(results)
backtester.print_performance_summary(metrics)

## 4. Benchmark Comparison

In [ ]:
# Calculate benchmark returns
benchmark_ticker = config['backtest']['benchmark']
benchmark_results = backtester.calculate_benchmark_returns(benchmark_ticker, prices)

if len(benchmark_results) > 0:
    benchmark_metrics = backtester.calculate_performance_metrics(benchmark_results)
    comparison = backtester.compare_to_benchmark(results, benchmark_results)
    
    print('\nBenchmark Performance:')
    print(f"Annualized Return: {benchmark_metrics['annualized_return']:.2%}")
    print(f"Sharpe Ratio: {benchmark_metrics['sharpe_ratio']:.2f}")
    print(f"Max Drawdown: {benchmark_metrics['max_drawdown']:.2%}")
    
    print('\nComparison:')
    print(f"Excess Return: {comparison['excess_return']:.2%}")
    print(f"Information Ratio: {comparison['information_ratio']:.2f}")
    print(f"Tracking Error: {comparison['tracking_error']:.2%}")

## 5. Performance Visualizations

In [ ]:
# Cumulative returns comparison
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(results.index, results['cumulative_returns'] * 100,
       label='Copy Congress Strategy', linewidth=2.5, color='blue')

if len(benchmark_results) > 0:
    ax.plot(benchmark_results.index, benchmark_results['cumulative_returns'] * 100,
           label=f'{benchmark_ticker} Benchmark', linewidth=2.5, color='gray', alpha=0.7)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cumulative Return (%)', fontsize=12)
ax.set_title('Copy Congress Strategy: Cumulative Performance', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Drawdown analysis
cumulative = (1 + results['returns']).cumprod()
running_max = cumulative.expanding().max()
drawdown = (cumulative - running_max) / running_max * 100

fig, ax = plt.subplots(figsize=(14, 7))
ax.fill_between(drawdown.index, drawdown, 0, alpha=0.3, color='red', label='Drawdown')
ax.plot(drawdown.index, drawdown, color='darkred', linewidth=1.5)
ax.axhline(y=config['risk']['max_drawdown'] * 100, color='orange', 
          linestyle='--', linewidth=2, label=f"Max DD Limit: {config['risk']['max_drawdown']:.0%}")
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Drawdown (%)', fontsize=12)
ax.set_title('Copy Congress Strategy: Drawdown Analysis', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Worst drawdown: {drawdown.min():.2f}%")
print(f"Current drawdown: {drawdown.iloc[-1]:.2f}%")

In [ ]:
# Rolling performance metrics
window = 252  # 1 year

rolling_returns = results['returns'].rolling(window=window)
rolling_sharpe = (rolling_returns.mean() / rolling_returns.std()) * np.sqrt(252)
rolling_vol = rolling_returns.std() * np.sqrt(252)

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Rolling Sharpe
axes[0].plot(rolling_sharpe.index, rolling_sharpe, linewidth=2, color='green')
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0].axhline(y=config['backtest']['target_sharpe'], color='red', 
               linestyle='--', label=f"Target: {config['backtest']['target_sharpe']:.2f}")
axes[0].set_ylabel('Sharpe Ratio', fontsize=11)
axes[0].set_title('1-Year Rolling Sharpe Ratio', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Rolling volatility
axes[1].plot(rolling_vol.index, rolling_vol * 100, linewidth=2, color='orange')
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_ylabel('Volatility (%)', fontsize=11)
axes[1].set_title('1-Year Rolling Volatility', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Returns distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Histogram
axes[0].hist(results['returns'].dropna() * 100, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(x=results['returns'].mean() * 100, color='red', 
               linestyle='--', linewidth=2, label=f"Mean: {results['returns'].mean()*100:.3f}%")
axes[0].set_xlabel('Daily Return (%)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Daily Returns', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Q-Q plot
from scipy import stats
stats.probplot(results['returns'].dropna(), dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot vs Normal Distribution', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate skewness and kurtosis
skew = results['returns'].dropna().skew()
kurt = results['returns'].dropna().kurtosis()
print(f"Skewness: {skew:.3f}")
print(f"Excess Kurtosis: {kurt:.3f}")

## 6. Monthly and Annual Returns

In [ ]:
# Monthly returns
monthly_returns = results['returns'].resample('M').apply(lambda x: (1 + x).prod() - 1)

print('Monthly Returns Summary:')
print(f"Average: {monthly_returns.mean():.2%}")
print(f"Median: {monthly_returns.median():.2%}")
print(f"Best month: {monthly_returns.max():.2%}")
print(f"Worst month: {monthly_returns.min():.2%}")
print(f"Win rate: {(monthly_returns > 0).sum() / len(monthly_returns):.1%}")

# Monthly returns bar chart
colors = ['green' if x > 0 else 'red' for x in monthly_returns]
plt.figure(figsize=(14, 6))
plt.bar(range(len(monthly_returns)), monthly_returns * 100, color=colors, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.xlabel('Month', fontsize=11)
plt.ylabel('Return (%)', fontsize=11)
plt.title('Monthly Returns', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Annual returns comparison
annual_returns = results['returns'].resample('Y').apply(lambda x: (1 + x).prod() - 1)

if len(benchmark_results) > 0:
    benchmark_annual = benchmark_results['returns'].resample('Y').apply(lambda x: (1 + x).prod() - 1)
    
    annual_comparison = pd.DataFrame({
        'Strategy': annual_returns,
        'Benchmark': benchmark_annual
    })
    
    annual_comparison['Excess'] = annual_comparison['Strategy'] - annual_comparison['Benchmark']
    
    print('\nAnnual Returns Comparison:')
    print((annual_comparison * 100).round(2))
    
    # Plot
    annual_comparison.plot(kind='bar', figsize=(12, 6), width=0.8)
    plt.xlabel('Year', fontsize=11)
    plt.ylabel('Return (%)', fontsize=11)
    plt.title('Annual Returns: Strategy vs Benchmark', fontsize=12, fontweight='bold')
    plt.legend(['Strategy', 'Benchmark', 'Excess'])
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

## 7. Portfolio Characteristics

In [ ]:
# Holdings over time
plt.figure(figsize=(14, 6))
plt.plot(results.index, results['n_positions'], linewidth=1.5, color='purple')
plt.axhline(y=config['portfolio']['min_holdings'], color='red', 
           linestyle='--', label=f"Min: {config['portfolio']['min_holdings']}")
plt.axhline(y=config['portfolio']['max_holdings'], color='orange', 
           linestyle='--', label=f"Max: {config['portfolio']['max_holdings']}")
plt.xlabel('Date', fontsize=11)
plt.ylabel('Number of Holdings', fontsize=11)
plt.title('Portfolio Holdings Over Time', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average holdings: {results['n_positions'].mean():.1f}")
print(f"Min holdings: {results['n_positions'].min()}")
print(f"Max holdings: {results['n_positions'].max()}")

## Summary

The backtest analysis provides comprehensive insights into:
- Absolute and risk-adjusted performance metrics
- Comparison to benchmark (SPY)
- Drawdown characteristics and risk management
- Return distribution and statistical properties
- Monthly and annual performance breakdown
- Portfolio composition dynamics